## 1. Parsing using mistune

In [ ]:
import os
import re
import time
import requests
import mistune
from bs4 import BeautifulSoup


# Custom renderer to treat HTML tags as plain text
class MyRenderer(mistune.Renderer):
    def text(self, text):
        return text

    def paragraph(self, text):
        return text + '\n\n'

    def block_code(self, code, lang=None):
        return '\n\n```\n{}\n```\n\n'.format(code)

    def block_quote(self, text):
        return '> {}\n\n'.format(text)

    def header(self, text, level, raw=None):
        return '{} {}\n\n'.format('#' * level, text)

    def list(self, body, ordered=True):
        return '{}\n\n'.format(body)

    def list_item(self, text):
        return '- {}\n'.format(text)

    def double_emphasis(self, text):
        return '**{}**'.format(text)

    def emphasis(self, text):
        return '*{}*'.format(text)

    def strikethrough(self, text):
        return '~~{}~~'.format(text)

    def linebreak(self):
        return '\n'

def clean_text(text):
    # Remove excessive line breaks and carriage returns
    cleaned_text = re.sub(r'[\r\n]+', '\n', text)
    cleaned_text = re.sub(r' +', ' ', text)
    cleaned_text = re.sub(r'^( +)', '', text, flags=re.MULTILINE)
    return cleaned_text

def parse_page(url):
    visited.add(url)
    print(f"Parsing page: {url}")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract text content and clean it
    text_content = soup.get_text()
    cleaned_content = clean_text(text_content)

    # Configure mistune with custom renderer
    markdown = mistune.Markdown(renderer=MyRenderer())

    # Convert cleaned text content to Markdown
    markdown_content = markdown(cleaned_content)

    # Save Markdown content to a file
    filename = url.replace('https://', '').replace('/', '_') + '.md'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(markdown_content)
        print(f"Markdown content saved to: {filename}")

    # Find links on the page and process them
    links = set()
    for link in soup.find_all('a', href=True):
        link_href = link.get('href')
        links.add(link_href)

        if (link_href.startswith('http') 
                and link_href not in visited
                and depth_dict.get(link_href, 0) < max_depth):
            queue.append(link_href)
            depth_dict[link_href] = depth_dict.get(url, 0) + 1

visited = set()
queue = ['https://innopolis.university/']
depth_dict = {'https://innopolis.university/': 0}
max_depth = 3

while queue:
    current_url = queue.pop(0)
    if current_url not in visited:
        parse_page(current_url)

In [ ]:
!cat innopolis.university_.md

## 2. Parsing using readability

### 2.1 Parsing

In [ ]:
!pip install readability-lxml

In [ ]:
from bs4 import BeautifulSoup

def extract_links_exclude_domains(html_content, base_url, excluded_domains):
    """
    Extract links from a webpage excluding specified domains.
    """
    soup = BeautifulSoup(html_content, 'html.parser')
    links = [link.get('href') for link in soup.find_all('a', href=True)]
    
    is_included = lambda x: not any(domain in x for domain in excluded_domains)
    is_innopolis = lambda x: 'innopolis' in x
    
    absolute_links = [urljoin(base_url, link) for link in links if is_included(link) and is_innopolis(link)]
    return absolute_links

In [12]:
import re
from readability import Document

def extract_summary(html_content):
    response = requests.get(html_content)
    doc = Document(response.content)
    soup = BeautifulSoup(doc.summary())
    summary = "\n".join([paragraph.strip() for paragraph in re.split("\n", soup.text) if len(paragraph)])
    return summary

In [ ]:
import os
import requests
from urllib.parse import urljoin

def crawl_webpage_exclude_domains(url, depth=0, max_depth=3, visited=None, excluded_domains=None):
    if visited is None:
        visited = set()
    if excluded_domains is None:
        excluded_domains = ['t.me', 'telegram.org', 'vk.com', 'youtube.com']

    try:
        if url in visited:
            return
        
        visited.add(url)
        print(f"Processing depth {depth}: {url}")
        
        response = requests.get(url)

        # Save content to a text file
        filename = f"{url.replace('/', '_').replace(':', '_')}.txt"
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(extract_readability_summary(url))

        # Extract links from the webpage excluding specified domains
        if depth < max_depth:
            links = extract_links_exclude_domains(response.content, url, excluded_domains)
            for link in links:
                crawl_webpage_exclude_domains(link, depth + 1, max_depth, visited, excluded_domains)
    except Exception as e:
        print(f"Error occurred: {e}")

In [15]:
starting_url = 'https://innopolis.university/'

if not os.path.exists('webpage_content'):
    os.makedirs('webpage_content')

os.chdir('webpage_content')

try:
    crawl_webpage_exclude_domains(starting_url, max_depth=1)
finally:
    os.chdir('..')

Processing depth 0: https://innopolis.university/
Processing depth 1: https://innopolis.university/about/
Processing depth 1: https://innopolis.university/board/
Processing depth 1: https://innopolis.university/team/
Processing depth 1: https://innopolis.university/team-structure/
Processing depth 1: https://innopolis.university/team-structure/education-academics/
Processing depth 1: https://innopolis.university/team-structure/techcenters/
Processing depth 1: https://innopolis.university/faculty/
Processing depth 1: https://career.innopolis.university/en/job/
Processing depth 1: https://career.innopolis.university
Processing depth 1: https://career.innopolis.university/job
Processing depth 1: https://career.innopolis.university/relocation/
Processing depth 1: https://innopolis.university/campus
Processing depth 1: https://innopolis.university/sveden/
Processing depth 1: http://iustore.innopolis.ru
Processing depth 1: https://apply.innopolis.university/
Processing depth 1: https://innop

### 2.2 Examples

In [21]:
print(extract_readability_summary('https://apply.innopolis.university/faq/'))

В качестве абитуриентов на бакалавриат мы ожидаем как граждан России, так и представителей зарубежных стран.
Граждане РФ:
— выпускники школ, завершающие обучение в 11 классе;
— выпускники колледжей с дипломами о среднем профессиональном образовании*;
— студенты выпускных классов колледжей ИТ-специальностей.
Граждане стран СНГ, поступающие в университет, также должны принадлежать к одной из вышеуказанных категорий. Сдавать ЕГЭ этим абитуриентам не требуется, они будут зачислены по результатам грантового конкурса и внутренних вступительных испытаний Университета Иннополис.
*Эта категория выпускников зачисляется при наличии выигранного гранта, диплома и действующих баллах ЕГЭ/по результатам внутренних вступительных испытаний Университета Иннополис.


In [24]:
print(extract_readability_summary('https://apply.innopolis.university/grant/'))

Первый шаг одинаков для всех – нужно зарегистрировать заявку (она же – личный кабинет абитуриента) на apply.innopolis.ru.
Главное - сделать это в нужный момент. Помимо заполнения анкеты, понадобится пройти проверку на уровень знания английского языка и IT-навыков.

Если результат рассмотрения заявки положительный, абитуриент приглашается на второй тур отборов.

В программе: оценка IT-навыков и знания английского языка, собеседование с профессорами на английском языке, оценка личностных качеств.

При успешном прохождении второго тура отборов будущий студент получает грант на обучение.


In [20]:
print(extract_readability_summary('https://media.innopolis.university/news/World-Science-Day/'))

Дмитрий Менделеев мастерил чемоданы, Никола Тесла любил голубей, Альберт Эйнштейн играл на скрипке, Лев Ландау увлекался кино, а Пьер и Мария Кюри путешествовали на велосипедах. Во Всемирный день науки сотрудники российского ИТ-вуза поделились мыслями о том, как их хобби помогают исследовательской работе и наоборот.
Валерия Скворцова, лаборант Центра технологий компонентов робототехники и мехатроники Университета Иннополис, преподаватель

В Университете Иннополис я занимаюсь разработкой экзоскелетов и реабилитационных девайсов. В целом, мне интересно создать новые приборы, оптимизировать дизайны под разные задачи и использовать робототехнику для восстановления или усиления моторных функций человека. Например, мои проекты — это робот для реабилитации кисти руки после инсульта и экзоскелет на базе приводов на скручивающихся нитях для компенсации усилий в теле человека при поднятии тяжестей с пола.

А после работы над роботами-манипуляторами и преподавания физики и мехатроники студентам я